In [1]:

import mysql.connector
from mysql.connector import Error

def create_connection():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='',
            database='attendance_db'
        )
        if connection.is_connected():
            print("✅ Connected to MySQL database")
            return connection
    except Error as e:
        print("❌ Error while connecting to MySQL:", e)
        return None

def insert_attendance(student_id, status):
    connection = create_connection()
    if connection:
        try:
            cursor = connection.cursor()
            query = "INSERT INTO attendance (student_id, status) VALUES (%s, %s)"
            values = (student_id, status)
            cursor.execute(query, values)
            connection.commit()
            print("✅ Attendance recorded for:", student_id)
        except Error as e:
            print("❌ Failed to insert attendance:", e)
        finally:
            connection.close()


In [2]:
import cv2

In [3]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        # scaling factor = 1.3
        # minimum neighbor = 5

        if faces is ():
            return None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h,x:x+w]
        return cropped_face

    cap = cv2.VideoCapture(0)
    id =1
    img_id = 0
    
    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame), (200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "data1/user."+str(id)+"."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            
            cv2.imshow("Cropped face", face)
            
        if cv2.waitKey(1)==13 or int(img_id)==400: #13 is the ASCII character of Enter
            break
            
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting samples is completed....")
#generate_dataset()

<>:9: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
<>:9: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
C:\Users\subed\AppData\Local\Temp\ipykernel_22204\3547298614.py:9: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
  if faces is ():


In [4]:
import os
from PIL import Image 
import numpy as np   

In [5]:
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image).convert('L') #converting to grayscale
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        
        faces.append(imageNp)
        ids.append(id)
        
    ids = np.array(ids)
    
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
#train_classifier("data1")

In [6]:
import dlib
from imutils import face_utils
from scipy.spatial import distance

In [ ]:
# Eye aspect ratio calculation
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def detect_with_liveness():
    # Thresholds and counters
    EYE_AR_THRESH = 0.23
    EYE_AR_CONSEC_FRAMES = 3
    COUNTER = 0
    BLINKED = False

    # Load required models
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read("classifier.xml")

    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)

        for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0

            if ear < EYE_AR_THRESH:
                COUNTER += 1
            else:
                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    BLINKED = True
                COUNTER = 0

            if BLINKED:
                faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                for (x, y, w, h) in faces:
                    id, pred = recognizer.predict(gray[y:y+h, x:x+w])
                    confidence = int(100 * (1 - pred/300))  # scale to percentage
                    if confidence > 75:
                        if id==1:
                            cv2.putText(frame, f"Name: Abhishek, ID: {id} ({confidence}%)", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
                    else:
                        cv2.putText(frame, "Unknown", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
                        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255), 2)
            else:
                cv2.putText(frame, "Please Blink", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

        cv2.imshow("Liveness + Recognition", frame)
        if cv2.waitKey(1) == 13:
            break

    cap.release()
    cv2.destroyAllWindows()

# To run this after training:
detect_with_liveness()


# Record attendance at the end of successful detection
insert_attendance(student_id="1234", status="Present")


import tkinter as tk
from tkinter import messagebox

#Shows success popup
root = tk.Tk()
root.withdraw()  # Hide main window
messagebox.showinfo("Attendance", "Attendance Successful!")
root.destroy()


✅ Connected to MySQL database
✅ Attendance recorded for: 1234
